In [6]:
from pathlib import Path
import datetime
import json
import gzip
import gc
from itertools import chain
from zoneinfo import ZoneInfo
import pandas as pd

# DATA PRE-PROCESSING

## BUILDING THE FINAL .csv FROM .json

* **Imports & configuration** – import `json`, `gzip`, `pathlib`, `itertools`, `pandas`, `tqdm`, `numpy`; set `RAW_DIR`, `CSV_DIR`, and the global `start_date`/`end_date`.

* **Raw JSON loading** – `_load_json_z` reads a single `.json.gz`; `_folder_to_df` concatenates all files in a folder into one DataFrame.

* **Timestamp construction** – `build_start_time` returns a UTC‐like `startTime` column, either from an existing field or by combining `settlementDate`+`settlementPeriod`.

* **Type coercion** – `_coerce_type` downcasts non-key columns to `float32` and normalises the date/time columns to naïve `datetime64[ns]`.

* **Small-gap interpolation** – `fill_small_gaps` linearly fills runs of up to two missing half-hours in numeric columns, leaving longer gaps untouched.

* **UK half-hour calendar** – `build_uk_halfhour_calendar` generates a full DST-aware sequence of half-hour intervals between any two dates.

* **Padding missing intervals** – `_pad_missing` merges data onto the full calendar (trimming to the global date range) so every expected interval appears.

* **Finalising pipeline** – `_finish` selects the requested columns, coerces types, pads missing, drops duplicates, interpolates small gaps, then sorts and resets the index.

* **Dataset builders** – each `b_<dataset>` function (e.g. `b_actual_demand`, `b_gen_per_type`, `b_system_prices`, etc.) computes `startTime`, selects its own `want` column list, and hands off to `_finish` to produce the final CSV.


In [46]:
# Set directories
RAW_DIR     = Path("bmrs_json_raw")
CSV_DIR     = Path("bmrs_csv_raw")
LOG_DIR     = Path("logs")

CSV_DIR.mkdir(exist_ok=True)
LOG_DIR.mkdir(exist_ok=True)


# Date range
start_date = "2017-01-01"
end_date   = "2025-06-30"



def _load_json_z(path: Path) -> list[dict]:
    with gzip.open(path, "rt") as fh:
        return json.load(fh)["data"]


def _folder_to_df(folder: Path) -> pd.DataFrame:
    files = sorted(folder.glob("*.json.gz"))
    rows  = chain.from_iterable((_load_json_z(f) for f in files))
    return pd.DataFrame.from_records(rows)


def build_start_time(df: pd.DataFrame) -> pd.Series:
    """Return UTC-like timestamp (start of settlement period)."""
    if "startTime" in df.columns:
        return pd.to_datetime(df["startTime"], errors="coerce")
    # otherwise compose from date + SP (SP1 = 00:00 UTC *winter*)
    base = pd.to_datetime(df["settlementDate"])
    off  = pd.to_timedelta(df["settlementPeriod"].astype(int).sub(1) * 30,
                           unit="m")
    return base + off




# ──────────────────────────────────────────────────────────────────────────
# ──────────────────────────────── finisher ────────────────────────────────
# ──────────────────────────────────────────────────────────────────────────

KEY_COLS = {"startTime", "settlementDate", "settlementPeriod"}

def _coerce_type(df: pd.DataFrame) -> pd.DataFrame:
    """
    • convert every non-key column to float32
    • normalise datetime columns
    """
    # 1) numeric columns  → float32
    num_cols = [c for c in df.columns if c not in KEY_COLS]
    df[num_cols] = df[num_cols].apply(
        pd.to_numeric, errors="coerce", downcast="float"
    )

    # 2) settlementDate  → 00:00 of that day, no timezone
    df["settlementDate"] = (
        pd.to_datetime(df["settlementDate"], utc=True)   # ensure tz-aware
          .dt.normalize()                                # strip hh:mm:ss
          .dt.tz_localize(None)                          # drop timezone
    )

    # 3) startTime  → no timezone (but keep hh:mm)
    if "startTime" in df.columns:
        df["startTime"] = (
            pd.to_datetime(df["startTime"], utc=True)
              .dt.tz_localize(None)
        )

    # settlementPeriod stays int32
    if "settlementPeriod" in df.columns:
        df["settlementPeriod"] = df["settlementPeriod"].astype("int32")

    return df

# ──────────────────────────────────────────────────────────────────────────

def build_uk_halfhour_calendar(start_date, end_date):
    """
    Build UK half-hour calendar with correct DST handling:
      • Spring-forward days: 46 periods (including the skipped 01:00/01:30)
      • Normal days: 48 periods 00:00-23:30
      • BST days: 48 periods 23:00(prev day)-22:30
      • Autumn-back days: 50 periods 23:00(prev day)-22:30
    """

    def _to_date(x):
        if isinstance(x, str):
            
            if x.count("-") == 2 and x[4] == "-": # ISO format
                return datetime.date.fromisoformat(x)
            return datetime.datetime.strptime(x, "%d/%m/%Y").date() # UK format
        if isinstance(x, pd.Timestamp):
            return x.date()
        return x

    start = _to_date(start_date)
    end   = _to_date(end_date)

    london = ZoneInfo("Europe/London")
    utc    = ZoneInfo("UTC")
    rows   = []

    for single in pd.date_range(start, end, freq="D"):
        D   = single.date()
        # local midnights in London
        dt0 = datetime.datetime(D.year, D.month, D.day, tzinfo=london)
        dt1 = dt0 + datetime.timedelta(days=1)

        # number of half-hours that actually occur
        total_secs = (dt1.astimezone(utc) - dt0.astimezone(utc)).total_seconds()
        n_periods = int(total_secs // 1800)

        # align to UTC-naive base for SP1
        offset_h = dt0.utcoffset().total_seconds() / 3600
        if offset_h > 0:
            base = datetime.datetime(D.year, D.month, D.day) - datetime.timedelta(hours=int(offset_h))
        else:
            base = datetime.datetime(D.year, D.month, D.day)

        for i in range(n_periods):
            rows.append({
                "startTime":        base + datetime.timedelta(minutes=30 * i),
                "settlementDate":   D,
                "settlementPeriod": i + 1
            })

    df = pd.DataFrame(rows)

    # ─── coerce to pandas time types ───
    df["startTime"]      = pd.to_datetime(df["startTime"])
    df["settlementDate"] = pd.to_datetime(df["settlementDate"]).dt.normalize()
    df["settlementPeriod"] = df["settlementPeriod"].astype("int32")
    # ───────────────────────────────────

    return df


# ──────────────────────────────────────────────────────────────────────────

def _pad_missing(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each settlementDate in df, build exactly the UK
    SP‐calendar via build_uk_halfhour_calendar(min,max), then
    left‐merge your data on (Date,Period,startTime).
    Also trims the input df to only include rows between start_date and end_date (inclusive).
    """
    if df.empty:
        return df

    df = df.copy()
    df=df.drop_duplicates(subset=["startTime"])
    # ensure proper types
    df["settlementDate"]   = pd.to_datetime(df["settlementDate"]).dt.normalize()
    df["settlementPeriod"] = df["settlementPeriod"].astype(int)
    df["startTime"]        = pd.to_datetime(df["startTime"])

    lo = start_date
    hi = end_date

    # trim input df to only include rows between start_date and end_date (inclusive)
    mask = (
        (df["settlementDate"] >= pd.to_datetime(start_date)) &
        (df["settlementDate"] <= pd.to_datetime(end_date))
    )
    df = df.loc[mask]

    # build the master calendar
    cal = build_uk_halfhour_calendar(lo, hi)
    cal["settlementDate"]   = pd.to_datetime(cal["settlementDate"])
    cal["settlementPeriod"] = cal["settlementPeriod"].astype(int)
    cal["startTime"]        = pd.to_datetime(cal["startTime"], dayfirst=True)

    # left‐join your actual data onto the calendar
    out = (
        cal
        .merge(df,
               on=["settlementDate","settlementPeriod","startTime"],
               how="left",
               sort=False)
    )

    return out

# ──────────────────────────────────────────────────────────────────────────

def _finish(out: pd.DataFrame, want: list[str]) -> pd.DataFrame:
    # 1) keep only requested columns
    out = out[want]
    # 2) downcast
    out = _coerce_type(out)
    # 3) pad missing with DST‐aware UK calendar
    out = _pad_missing(out)
    # 4) drop duplicate rows
    out = out.drop_duplicates()
    
    return out.sort_values("startTime").reset_index(drop=True)




# ──────────────────────────────────────────────────────────────────────────
# ──────────────────────────────── builders ────────────────────────────────
# ──────────────────────────────────────────────────────────────────────────

def b_actual_demand(df):
    df["startTime"] = build_start_time(df)
    want = ["startTime","settlementDate","settlementPeriod",
            "initialDemandOutturn",
            "initialTransmissionSystemDemandOutturn"]
    out = df[want]
    return _finish(out, want)

# ──────────────────────────────────────────────────────────────────────────
def _pivot_wind_solar(df, value_name):
    map_ = {
        "Wind Onshore" : "windOnshoreGeneration",
        "Wind Offshore": "windOffshoreGeneration",
        "Solar"        : "solarGeneration",
    }
    df  = df.replace({"psrType": map_})
    out = (df.pivot_table(index=["settlementDate","settlementPeriod","startTime"],
                          columns="psrType",
                          values=value_name, aggfunc="first")
             .reset_index())
    out["startTime"] = build_start_time(out)
    for c in map_.values():
        if c not in out.columns:
            out[c] = pd.NA
    return out

def b_actual_gen_ws(df):
    tidy = _pivot_wind_solar(df.rename(columns={"quantity":"gen"}), "gen")
    want = ["startTime","settlementDate","settlementPeriod",
            "windOnshoreGeneration","windOffshoreGeneration",
            "solarGeneration"]
    out = tidy[want]
    return _finish(out, want)

# ──────────────────────────────────────────────────────────────────────────

def _pivot_wind_solar_forecast(df, process_type, value_name):
    map_ = {
        "Wind Onshore":  "windOnshoreGeneration",
        "Wind Offshore": "windOffshoreGeneration",
        "Solar":         "solarGeneration",
    }
    df = (
        df.loc[df["processType"] == process_type]
          .rename(columns={"quantity": value_name})
          .replace({"psrType": map_})
    )
    out = (
        df.pivot_table(
            index=["settlementDate", "settlementPeriod", "startTime", "publishTime"],
            columns="psrType",
            values=value_name,
            aggfunc="first"
        )
        .reset_index()
    )
    out["startTime"] = build_start_time(out)
    for col in map_.values():
        if col not in out.columns:
            out[col] = pd.NA
    return out

def b_dayahead_gen_ws(df):
    tidy = _pivot_wind_solar_forecast(df, "Day ahead", "forecast")
    want = [
        "startTime", "settlementDate", "settlementPeriod", "publishTime",
        "windOnshoreGeneration", "windOffshoreGeneration", "solarGeneration"
    ]
    return _finish(tidy[want], want)

def b_intradayprocess_gen_ws(df):
    tidy = _pivot_wind_solar_forecast(df, "Intraday process", "forecast")
    want = [
        "startTime", "settlementDate", "settlementPeriod", "publishTime",
        "windOnshoreGeneration", "windOffshoreGeneration", "solarGeneration"
    ]
    return _finish(tidy[want], want)

def b_intradaytotal_gen_ws(df):
    tidy = _pivot_wind_solar_forecast(df, "Intraday total", "forecast")
    want = [
        "startTime", "settlementDate", "settlementPeriod", "publishTime",
        "windOnshoreGeneration", "windOffshoreGeneration", "solarGeneration"
    ]
    return _finish(tidy[want], want)

# ──────────────────────────────────────────────────────────────────────────

def _folder_to_df_detailed(folder: Path) -> pd.DataFrame:
    """
    Like _folder_to_df, but also injects `publishSP` from each filename:
      bmrs_json_raw/DETAILED_WINDFOR/YYYY-MM-DD_<SP>.json.gz
    """
    rows = []
    for f in sorted(folder.glob("*.json.gz")):
        name = f.name  # e.g. "2017-01-01_10.json.gz"
        # strip off the ".json.gz" bit, then split on "_"
        base = name[:-len(".json.gz")]            # → "2017-01-01_10"
        sp   = int(base.split("_", 1)[1])         # → 10
        # … load JSON …
        payload = json.load(gzip.open(f, "rt"))
        for rec in payload.get("data", []):
           rec["publishSP"] = sp
           rows.append(rec)
    return pd.DataFrame.from_records(rows)

def b_windfor_evolution(df):
    """
    Intraday wind forecasts with 8 columns f_1…f_8,
    where f_1 is the latest run ≥1h before delivery,
    f_2 the next‐latest, …, f_8 the 8th‐latest.
    """
    want = ["startTime","settlementDate","settlementPeriod"] + [f"f_{i}" for i in range(1,9)]

    if df.empty:
        return pd.DataFrame(columns=want)

    # 1) parse
    df["publishTimeUTC"] = pd.to_datetime(df["publishTime"], utc=True)
    df["startTime"]      = pd.to_datetime(df["startTime"],   utc=True)

    # 2) gate‐closure: only keep runs published ≥1h before delivery
    df = df[df["publishTimeUTC"] <= df["startTime"] - pd.Timedelta(hours=1)]

    # 3) sort by slot & publishTime descending
    df = df.sort_values(
        ["settlementDate","settlementPeriod","startTime","publishTimeUTC"],
        ascending=[True, True, True, False]
    )

    # 4) rank each slot’s forecasts 1…n by recency
    df["rank"] = (
        df
        .groupby(["settlementDate","settlementPeriod","startTime"])
        .cumcount()
        + 1
    )

    # 5) keep only the top 8 for each slot
    df8 = df[df["rank"] <= 8]

    # 6) pivot so rank → f_<rank>
    wide = (
        df8.pivot_table(
            index=["settlementDate","settlementPeriod","startTime"],
            columns="rank",
            values="generation",
            aggfunc="first"
        )
        .reset_index()
    )

    # 7) rename and ensure f_1…f_8 exist
    for i in range(1,9):
        col = f"f_{i}"
        if i in wide.columns:
            wide = wide.rename(columns={i: col})
        else:
            wide[col] = pd.NA

    # 8) rebuild startTime (UTC→naive half‐hour)
    wide["startTime"] = build_start_time(wide)

    # 9) finish (dtype coercion, calendar padding, dedupe, sort)
    return _finish(wide[want], want)


def b_windfor(df):
    """
    Intraday wind forecasts pivoted into wind_f1…wind_f48,
    indexed purely by publish‐time SP, with calendar‐merge
    for settlementDate/settlementPeriod.
    """
    want = ["startTime","settlementDate","settlementPeriod"] \
         + [f"wind_f{i}" for i in range(1,49)]
    if df.empty:
        return pd.DataFrame(columns=want)

    # 1) parse timestamps
    df["publishTime"]  = pd.to_datetime(df["publishTime"])
    df["deliveryTime"] = pd.to_datetime(df["startTime"])

    # 2) use publishTime as our row index
    df["startTime"] = df["publishTime"]

    # 3) compute horizon = # half-hours ahead
    df["horizon"] = ((df["deliveryTime"] - df["publishTime"])
                      .dt.total_seconds() // 1800).astype(int)

    # 4) pivot only on startTime
    wide = (
      df
      .pivot_table(
          index="startTime",
          columns="horizon",
          values="generation",
          aggfunc="first"
      )
      .reset_index()
      .rename(columns={i: f"wind_f{i}" for i in range(1,49)})
    )

    wide["startTime"] = wide["startTime"].dt.tz_convert(None)

    # 5) bring in settlementDate & settlementPeriod via full calendar
    cal = build_uk_halfhour_calendar(start_date, end_date)
    wide = cal.merge(wide, on="startTime", how="left")

    # 6) ensure exactly wind_f1…wind_f48 exist
    for i in range(1,49):
        col = f"wind_f{i}"
        if col not in wide.columns:
            wide[col] = pd.NA

    # 7) coerce types, pad missing SPs, drop duplicates, sort
    return _finish(wide[want], want)

# ──────────────────────────────────────────────────────────────────────────
def b_dayahead_demand(df):
    df["startTime"] = build_start_time(df)
    want = ["startTime", "settlementDate", "settlementPeriod",
            "transmissionSystemDemand", "nationalDemand"]
    out = df[want]
    return _finish(out, want)


# ──────────────────────────────────────────────────────────────────────────
def b_indicated(df):
    df["startTime"] = build_start_time(df)
    want = ["startTime", "settlementDate", "settlementPeriod",
            "indicatedGeneration", "indicatedDemand",
            "indicatedMargin", "indicatedImbalance"]
    out = df[want]
    return _finish(out, want)


# ──────────────────────────────────────────────────────────────────────────
IC_NAME_TO_COL = {
    "Eleclink (INTELEC)"      : "INTELEC",
    "Ireland(East-West)"      : "INTEW",
    "France(IFA)"             : "INTFR",
    "Ireland (Greenlink)"     : "INTGRNL",
    "IFA2 (INTIFA2)"          : "INTIFA2",
    "Northern Ireland(Moyle)" : "INTIRL",
    "Netherlands(BritNed)"    : "INTNED",
    "Belgium (Nemolink)"      : "INTNEM",
    "North Sea Link (INTNSL)" : "INTNSL",
    "Denmark (Viking link)"   : "INTVKL",
}

def b_inter(df):
    df = df.replace({"interconnectorName": IC_NAME_TO_COL})

    df = (df.pivot_table(index=["settlementDate", "settlementPeriod", "startTime"],
                         columns="interconnectorName",
                         values="generation",
                         aggfunc="first")
            .reset_index())

    df["startTime"] = build_start_time(df)

    for col in IC_NAME_TO_COL.values():
        if col not in df.columns:
            df[col] = pd.NA
    

    want = ["startTime", "settlementDate", "settlementPeriod"] + list(IC_NAME_TO_COL.values())
    out = df[want]
    return _finish(out, want)


# ──────────────────────────────────────────────────────────────────────────
def b_mid(df):
    df = df.loc[df["dataProvider"] == "APXMIDP"].copy()
    df["startTime"] = build_start_time(df)
    want = ["startTime", "settlementDate", "settlementPeriod",
            "price", "volume"]
    return _finish(df[want], want)

# ──────────────────────────────────────────────────────────────────────────
def b_nonbm(df):
    df["startTime"] = build_start_time(df)
    want = ["startTime", "settlementDate", "settlementPeriod",
            "generation"]
    return _finish(df[want], want)

# ──────────────────────────────────────────────────────────────────────────
HORIZONS = [1, 2, 4, 8, 12]

def b_lolpdrm(df):
    df["startTime"] = build_start_time(df)

    # keep only horizons we care about
    df = df.loc[df["forecastHorizon"].isin(HORIZONS),
                ["startTime", "settlementDate", "settlementPeriod",
                 "forecastHorizon", "lossOfLoadProbability",
                 "deratedMargin"]]

    # ----------  LOLP (horizon 1)  ----------
    lolp = (df[df["forecastHorizon"] == 1].rename(columns={"lossOfLoadProbability": "1hLOLP"}).loc[:, ["startTime", "settlementDate",
                       "settlementPeriod", "1hLOLP"]])

    # ----------  DRM (pivot all horizons)  ----------
    drm = (df.pivot_table(index=["startTime", "settlementDate",
                                 "settlementPeriod"],
                          columns="forecastHorizon",
                          values="deratedMargin")
            .rename(columns={h: f"{h}hDRM" for h in HORIZONS})
            .reset_index())

    # ----------  merge & order columns  ----------
    out = lolp.merge(drm, on=["startTime", "settlementDate",
                              "settlementPeriod"])

    want = ["startTime", "settlementDate", "settlementPeriod",
            "1hLOLP", "1hDRM"]
    out = out[want]
    return _finish(out, want)


def b_lolpdrm_sp(df: pd.DataFrame) -> pd.DataFrame:
    """
    Pivot de-rated margin for settlement-period horizons 1…48 into drm_f1…drm_f48,
    indexed by publishingPeriodCommencingTime.
    """
    # target columns
    drm_cols = [f"drm_f{i}" for i in range(1,49)]
    want     = ["startTime", "settlementDate", "settlementPeriod"] + drm_cols

    if df.empty:
        return pd.DataFrame(columns=want)

    # 1) parse times
    df["publishTime"] = pd.to_datetime(df["publishingPeriodCommencingTime"], utc=True)
    df["deliveryTime"] = pd.to_datetime(df["startTime"], utc=True)

    # 2) use the exact publish-period time as our row index
    df["startTime"] = df["publishTime"].dt.tz_localize(None)

    # 3) compute half-hour horizon (in SPs)
    df["horizon_sp"] = (
        (df["deliveryTime"] - df["publishTime"])
        .dt.total_seconds() // 1800
    ).astype(int)

    # 4) pivot ALL horizons 1…48
    wide = (
        df
        .pivot_table(
            index="startTime",
            columns="horizon_sp",
            values="deratedMargin",
            aggfunc="first",
        )
        .reset_index()
        .rename(columns={i: f"drm_f{i}" for i in range(1,49)})
    )

    # 5) full calendar merge to recover settlementDate/Period
    cal = build_uk_halfhour_calendar(start_date, end_date)
    wide = cal.merge(wide, on="startTime", how="left")

    # 6) ensure drm_f1…drm_f48 all exist
    for col in drm_cols:
        if col not in wide.columns:
            wide[col] = pd.NA

    # 7) final cleanup: dtype coercion, calendar-pad, dedupe, sort
    return _finish(wide[want], want)


# ──────────────────────────────────────────────────────────────────────────
def b_system_prices(df):
    df["startTime"] = build_start_time(df)

    df = df.rename(columns={
        "systemSellPrice"    : "systemPrice",      # SSP / SBP
        "netImbalanceVolume" : "netImbalanceVolume"
    })

    want = ["startTime", "settlementDate", "settlementPeriod",
            "systemPrice", "netImbalanceVolume",
            "sellPriceAdjustment", "buyPriceAdjustment",
            "replacementPrice", "replacementPriceReferenceVolume",
            "totalAcceptedOfferVolume", "totalAcceptedBidVolume",
            "totalAdjustmentSellVolume", "totalAdjustmentBuyVolume",
            "totalSystemTaggedAcceptedOfferVolume",
            "totalSystemTaggedAcceptedBidVolume",
            "totalSystemTaggedAdjustmentSellVolume",
            "totalSystemTaggedAdjustmentBuyVolume"]

    # create any missing columns so _finish keeps dtype order
    for col in want:
        if col not in df.columns:
            df[col] = pd.NA

    out = df[want]
    return _finish(out, want)


In [47]:
# BUILDERS = {
#     "ACTUAL_DEMAND"                  : b_actual_demand,
#     "ACTUAL_GEN_WIND_SOLAR"          : b_actual_gen_ws,
#     "DAYAHEAD_DEMAND"                : b_dayahead_demand,
#     "DETAILED_WINDFOR"               : b_windfor,
#     "DAYAHEAD_GEN_WIND_SOLAR"        : b_dayahead_gen_ws,
#     "INTRADAYPROCESS_GEN_WIND_SOLAR" : b_intradayprocess_gen_ws,
#     "INTRADAYTOTAL_GEN_WIND_SOLAR"   : b_intradaytotal_gen_ws,
#     "GEN_PER_TYPE"                   : b_gen_per_type,
#     "INDICATED_DAYAHEAD_DEMAND"      : b_indicated,
#     "INTER"                          : b_inter,
#     "LOLPDRM"                        : b_lolpdrm,
#     "NONBM"                          : b_nonbm,
#     "MID"                            : b_mid,
#     "SYSTEM_PRICES"                  : b_system_prices,
# }

BUILDERS = {
    "DRM_FORECASTS"               : b_lolpdrm_sp,
}


def process_one(code: str, builder):
    # Map output codes to their real input folders
    if code == "DRM_FORECASTS":
        folder = RAW_DIR / "LOLPDRM"
    elif code == "WIND_FORECASTS":
        folder = RAW_DIR / "WINDFOR_HISTORY"
    else:
        folder = RAW_DIR / code

    print(f"\n→ Looking in folder: {folder}")
    files = sorted(folder.glob("*.json.gz"))
    print(f"   Found {len(files)} files: {files[:5]}{'…' if len(files)>5 else ''}")

    if "WINDFOR" in folder.name:
        df_raw = _folder_to_df_detailed(folder)
    else:
        df_raw = _folder_to_df(folder)

    print(f"   Loaded raw rows: {len(df_raw)}")
    if df_raw.empty:
        print(f"⚠ {code}: no data, skipping")
        return None

    df_tidy = builder(df_raw)
    print(f"   Builder produced {len(df_tidy)} rows")

    out = CSV_DIR / f"{code}.csv"
    df_tidy.to_csv(out, index=False)
    print(f"✓ {code}: wrote {len(df_tidy):,} rows → {out}")
    return df_tidy



def main():
    # 1) run all builders and write CSVs
    for code, builder in BUILDERS.items():
        result = process_one(code, builder)

if __name__ == "__main__":
    pd.set_option("future.no_silent_downcasting", True)
    main()



→ Looking in folder: bmrs_json_raw/LOLPDRM
   Found 3104 files: [PosixPath('bmrs_json_raw/LOLPDRM/2017-01-01.json.gz'), PosixPath('bmrs_json_raw/LOLPDRM/2017-01-02.json.gz'), PosixPath('bmrs_json_raw/LOLPDRM/2017-01-03.json.gz'), PosixPath('bmrs_json_raw/LOLPDRM/2017-01-04.json.gz'), PosixPath('bmrs_json_raw/LOLPDRM/2017-01-05.json.gz')]…
   Loaded raw rows: 737014
   Builder produced 148942 rows
✓ DRM_FORECASTS: wrote 148,942 rows → bmrs_csv_raw/DRM_FORECASTS.csv
